# Dask Gateway Example Notebook
Here are some commands for starting and working with a Dask Gateway cluster.

## Start Dask cluster

In [1]:
import os
import dask
from dask_gateway import Gateway
from dask.distributed import Client
import dask_gateway
import dask.array as da

In [2]:
# get the dask-gateway version
dask_gateway.__version__

'0.7.1'

In [3]:
# show the default dask-gateway settings
dask.config.config['gateway']

{'auth': {'type': 'jupyterhub', 'kwargs': {}},
 'cluster': {'options': {'image': '{JUPYTER_IMAGE_SPEC}'}},
 'address': 'https://staging.ooi.pangeo.io/services/dask-gateway/',
 'proxy_address': 'gateway://traefik-ooi-staging-dask-gateway.ooi-staging:80',
 'public-address': None,
 'http-client': {'proxy': True}}

In [4]:
# show the current default image to be started on workers
os.environ['JUPYTER_IMAGE_SPEC']

'ooicloud.azurecr.io/ooi-pangeo-io-notebook:1590ac8'

In [5]:
# explicit gateway call
#gateway = Gateway(address = 'https://staging.ooi.pangeo.io/services/dask-gateway/',
#                  proxy_address = 'gateway://traefik-ooi-staging-dask-gateway.ooi-staging:80',
#                  auth = 'jupyterhub')

In [6]:
#default gateway call
gateway = Gateway()

In [7]:
# call new_cluster with explicit image
#cluster = gateway.new_cluster(image=os.environ['JUPYTER_IMAGE'])

In [8]:
# default new_cluster call
cluster = gateway.new_cluster()

In [9]:
# get the name of the new cluster
gateway.list_clusters()[0].name

'ooi-staging.cf96bf958a0647739ed95448721d0b35'

In [10]:
# connect to cluster that already exists
# this is helpful if the cluster exists but it is disconnected for some reason
#cluster_name = gateway.list_clusters()[0].name
#cluster = gateway.connect(cluster_name)

In [11]:
# the dashboard_link property will show the link that can be pasted into the Dask labextension
cluster.dashboard_link

'https://staging.ooi.pangeo.io/services/dask-gateway/clusters/ooi-staging.cf96bf958a0647739ed95448721d0b35/status'

## Scale cluster and connect a client
"cluster" will show the interactive scaling controls and the dashboard link, which can be pasted into the Dask labextension.

In [12]:
cluster

In [13]:
# scale cluster to 4 workers using the scale() method
cluster.scale(4)

In [14]:
# connect a client
# the distributed client is used for running parallel tasks with Dask
client = Client(cluster)

In [15]:
client

Client Scheduler: gateway://traefik-ooi-staging-dask-gateway.ooi-staging:80/ooi-staging.cf96bf958a0647739ed95448721d0b35 Dashboard: https://staging.ooi.pangeo.io/services/dask-gateway/clusters/ooi-staging.cf96bf958a0647739ed95448721d0b35/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


## Test Dask cluster

In [16]:
%%time
a = da.random.normal(size=(10000, 100000), chunks=(500, 500))
f = client.submit(a.mean().compute)
f.result()

CPU times: user 429 ms, sys: 56.4 ms, total: 485 ms
Wall time: 18.6 s


-6.609074951680086e-06

## Other misc commands
These commands are useful for getting information but aren't normally needed

In [17]:
options = gateway.cluster_options()

In [18]:
options

Box(children=(Box(children=(HTML(value='<h2>Cluster Options</h2>'), Box(children=(HTML(value="<p style='font-w…

In [19]:
options.worker_cores = 4

In [20]:
client.scheduler_info()['services']

{'dashboard': 8787, 'gateway': 8788}

In [21]:
#cluster.get_client()

In [22]:
gateway.address

'https://staging.ooi.pangeo.io/services/dask-gateway'

In [23]:
cluster.scheduler_address

'gateway://traefik-ooi-staging-dask-gateway.ooi-staging:80/ooi-staging.cf96bf958a0647739ed95448721d0b35'

In [24]:
cluster.scheduler_comm

<rpc to 'gateway://traefik-ooi-staging-dask-gateway.ooi-staging:80/ooi-staging.cf96bf958a0647739ed95448721d0b35', 1 comms>

In [25]:
cluster.scheduler_info

{'type': 'Scheduler',
 'id': 'Scheduler-c5d0ab85-ab85-49ff-bfce-81cd8f0345a7',
 'address': 'tls://10.0.130.193:8786',
 'services': {'dashboard': 8787, 'gateway': 8788},
 'workers': {'tls://10.0.129.112:40357': {'type': 'Worker',
   'id': 'dask-gateway-nwt5r',
   'host': '10.0.129.112',
   'resources': {},
   'local_directory': '/home/jovyan/worker-ehx3jot4',
   'name': 'dask-gateway-nwt5r',
   'nthreads': 2,
   'memory_limit': 4294967296,
   'services': {'dashboard': 45015},
   'nanny': 'tls://10.0.129.112:37095'},
  'tls://10.0.129.30:34303': {'type': 'Worker',
   'id': 'dask-gateway-s5x6v',
   'host': '10.0.129.30',
   'resources': {},
   'local_directory': '/home/jovyan/worker-91ov5ybt',
   'name': 'dask-gateway-s5x6v',
   'nthreads': 2,
   'memory_limit': 4294967296,
   'services': {'dashboard': 40761},
   'nanny': 'tls://10.0.129.30:34823'},
  'tls://10.0.128.12:33021': {'type': 'Worker',
   'id': 'dask-gateway-z4rp5',
   'host': '10.0.128.12',
   'resources': {},
   'local_direct

## Shutdown cluster

In [ ]:
cluster.shutdown()